## [data.go.kr 실습] 국토부 아파트매매 실거래가 API 연동
#### https://www.data.go.kr/data/15058747/openapi.do
#### https://www.code.go.kr/index.do

In [1]:
in_svckey = "fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D"

### 1. Import Packages

In [2]:
import requests
import pandas as pd

### 2. Data Loading

In [3]:
df_lawcd = pd.read_csv("법정동코드 전체자료.txt", encoding="cp949", sep="\t")

In [4]:
df_lawcd.shape

(46314, 3)

In [5]:
df_lawcd.head(3)

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재


In [ ]:
in_lawd_name = input("동네명 입력 : ")

In [ ]:
in_deal_ymd = input("거래계약 월 (예: 202202) : ")

In [ ]:
df_lawcd.head(3)
df_lawcd[df_lawcd["법정동명"].str.contains(in_lawd_name)]

In [ ]:
df_lawcd[df_lawcd["폐지여부"]=="존재"]

In [ ]:
tmp = df_lawcd[(df_lawcd["법정동명"].str.contains(in_lawd_name)) & (df_lawcd["폐지여부"]=="존재")]

In [ ]:
tmp

In [ ]:
str(tmp.iloc[0, 0])[:5]

#앞에 값 보고싶으면 index, 뒤에 값 보고싶으면 values
# iloc쓸라면 무조건 인덱스 써야됨

In [ ]:
def find_lawdcd(in_lawd_name):
    tmp = df_lawcd[(df_lawcd["법정동명"].str.contains(in_lawd_name)) & (df_lawcd["폐지여부"]=="존재")]
    return str(tmp.iloc[0, 0])[:5]

In [ ]:
in_lawd_name = input("동네명 입력 : ")
in_deal_ymd = input("거래계약 월 (예: 202202) : ")

in_lawd_cd = find_lawdcd(in_lawd_name)
in_lawd_cd

### 3. Input Data : 법정코드, 거래년월

In [ ]:
base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?"

In [ ]:
lawd_cd = "LAWD_CD=" + in_lawd_cd #"11110" #&
deal_ymd = "DEAL_YMD=" + in_deal_ymd #"202106" #&
svc_key = "serviceKey=" + in_svckey

In [ ]:
url = base_url + lawd_cd + "&" + deal_ymd + "&" + svc_key
url

# 아래 나온 url 인터넷 주소창에 한번 쳐서 잘 됐는지 학인

In [ ]:
response = requests.get(url)
response

# 200이 정상, 나머지 비정상

### 4. Data Preprocessing

In [ ]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

In [ ]:
soup = BeautifulSoup(response.content, "lxml")
#soup.find('items')
# HTML은 예정된 키워드 외에는 못 써 (ex <a>, <p>, ...)
# XML은 HTML의 확장판 -> 태그를 내가 만들 수 있음 (ex. <주소>, </주소>, <거래 금액>)

In [ ]:
# XML 용 parsing function :

def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [ ]:
result = pd.DataFrame(find_xml_items(response))
result.head()

### 5. Save as a file

In [ ]:
#예시: 강북구_202012-20210824.csv 

import time
tdate = time.strftime('%x', time.localtime(time.time())).replace('/','')
tdate

In [ ]:
in_lawd_name
in_deal_ymd

In [ ]:
file_name = in_lawd_name + "-" + in_deal_ymd + "-" + tdate + ".csv"
file_name

In [ ]:
file_name = tdate + ".csv"
file_name

In [ ]:
result.to_csv(file_name, index=False, encoding="cp949")